# Let's train a LSTM to write Songs!

We're going to use [Keras](https://keras.io)  to write songs. At least 20 epochs are required before the generated text starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.


### Let's check out the corpus

In [ ]:
from __future__ import print_function
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys

# Read the entire file containing nietzsche's works
path = './data/songdata.csv'

df = pd.read_csv(path)
df.tail()

In [ ]:
text = df['text'].str.cat(sep='\n').lower()
# Output the length of the corpus
print('corpus length:', len(text))


# Create a sorted list of the characters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# Corpus is going to take tooooo long to train, so lets make it shorter
text = text[:1000000]
print('truncated corpus length:', len(text))

# Create a dictionary where given a character, you can look up the index and vice versa
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3

## Creates the overlapping windows with target characters

In [ ]:

sentences = []
next_chars = []

# Step through the text via 3 characters at a time, taking a sequence of 40 bytes at a time. 
# There will be lots ofo overlap
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen]) # range from current index i for max length characters 
    next_chars.append(text[i + maxlen]) # the next character after that 
sentences = np.array(sentences)
next_chars = np.array(next_chars)
print('Number of sequences:', len(sentences))

## Generates the 1 hot vectors for each character

In [5]:

def generator(sentences, next_chars, batch_size):
    X = np.zeros((batch_size, maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((batch_size, len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    while True:
        if index + batch_size >= length:
            index = 0
        X.fill(0)
        y.fill(0)
        for i in range(batch_size):
            sentence = sentences[index]
            for t, char in enumerate(sentence):
                X[i, t, char_indices[char]] = 1
            y[i, char_indices[next_chars[i]]] = 1
            index = index + 1
        yield X, y

        
def getdata(sentences, next_chars):
    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

## Build the LSTM model

In [ ]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print("Compiling model complete...")


### Helper function to sample an index from a probability array
 The purpose of this function is to add some randomness so that the most likely character is not always chosen, and sometiems the 2nd or 3rd most likely cahracter is chosen

In [6]:

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### And now the actual training...

In [19]:
diversity = 0.25
print('Diversity: ', diversity)

# Get data
X, y = getdata(sentences, next_chars)

# The training
print('Training...')
batch_size = 128
#history = model.fit_generator(generator(sentences, next_chars, batch_size),steps_per_epoch=12800, epochs=10)

history = model.fit(X, y,batch_size=128, epochs=30)


# Save the model
model.save('songgenerator.h5')

Diversity:  0.25
Training...
Epoch 1/30
333320/333320 [==============================] - 90s - loss: 1.3257    
Epoch 2/30
333320/333320 [==============================] - 89s - loss: 1.3079    
Epoch 3/30
333320/333320 [==============================] - 89s - loss: 1.2916    
Epoch 4/30
333320/333320 [==============================] - 89s - loss: 1.2767    
Epoch 5/30
333320/333320 [==============================] - 95s - loss: 1.2633    
Epoch 6/30
333320/333320 [==============================] - 91s - loss: 1.2509    
Epoch 7/30
333320/333320 [==============================] - 90s - loss: 1.2396    
Epoch 8/30
333320/333320 [==============================] - 90s - loss: 1.2289    
Epoch 9/30
333320/333320 [==============================] - 92s - loss: 1.2182    
Epoch 10/30
333320/333320 [==============================] - 89s - loss: 1.2090    
Epoch 11/30
333320/333320 [==============================] - 93s - loss: 1.2000    
Epoch 12/30
333320/333320 [=============================

In [7]:
# Save the model
model = load_model('songgenerator.h5')

In [ ]:
# Check out what our model predicts
sentence = 'behold, my field of cares\nalas, but there is nothing'
sentence = 'familiar like family\nancient it\'s gravity'
sentence = sentence[:40]
x = np.zeros((1, maxlen, len(chars)))
for t, char in enumerate(sentence):
    x[0, t, char_indices[char]] = 1.
    
print(model.predict(x, verbose=0)[0])
print(sum(model.predict(x, verbose=0)[0]))

In [ ]:
generated = ''
original = sentence
# Predict the next 400 characters based on the seed
for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.2)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

print(original + generated)
